Find all TCP and UDP streams sequence numbers from pcap

In [ ]:
%%bash 
tshark -r Monday-WorkingHours_DeDup.pcap -Y "tcp || udp" -T fields -e tcp.stream -e udp.stream | sort | uniq > monday_all_streams.txt

Take 1% of the total TCP and UDP streams sequence numbers from pcap

In [ ]:
%%bash 
total_streams=$(wc -l < monday_all_streams.txt)
select_count=$((total_streams * 3 / 100))  # Adjust the percentage here
shuf monday_all_streams.txt | head -n $select_count > monday_selected_streams.txt

Extract the selected TCP and UDP streams from pcap

In [ ]:
%%bash
output_file="Benign_10per_streams.pcap"
> $output_file

# Determine chunk size, e.g., 100 streams per chunk
chunk_size=500
total_lines=$(cat monday_selected_streams.txt | wc -l)
num_chunks=$((total_lines / chunk_size + (total_lines % chunk_size > 0)))

for ((i = 1; i <= num_chunks; i++)); do
    start_line=$(( (i - 1) * chunk_size + 1 ))
    end_line=$(( i * chunk_size ))
    
    filter=""
    for stream in $(sed -n "${start_line},${end_line}p" monday_selected_streams.txt); do
        filter="${filter:+$filter || }tcp.stream==$stream || udp.stream==$stream"
    done
    
    tshark -r Monday-WorkingHours_DeDup.pcap -Y "$filter" -w "Benign_chunk_${i}.pcap"
done

# Merge all chunks
mergecap -w $output_file Benign_chunk_*.pcap

# Cleanup
rm Benign_chunk_*.pcap